In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import model_utils as mu
from gensim.models import Word2Vec
import numpy as np
import torch
import torch.nn as nn
from tqdm.autonotebook import tqdm
import nn_model as nnm
from sklearn.preprocessing import LabelEncoder

NUM_SEQUENCES_PER_BATCH = 512

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
# Load the pre-trained model
embedder = nnm.Embedder('all-MiniLM-L6-v2', device=device)

In [3]:
#load sentences
similar_song_lyrics= pd.read_csv("data/kat_data_similar_songs.csv") 
similar_song_lyrics.head()


,title,artist,lyrics,similar_track,similar_artist,similar_song
0,Cuéntale,Ivy Queen,4 ContributorsDile LyricsQuien te va a querer ...,Dile,Ivy Queen,Dile by Ivy Queen
1,Cuéntale,Ivy Queen,3 ContributorsCansada LyricsWelcome to the dra...,Cansada,Ivy Queen,Cansada by Ivy Queen
2,Cuéntale,Ivy Queen,11 ContributorsTranslationsEnglishDeutschElla ...,Ella Me Levantó,Daddy Yankee,Ella Me Levantó by Daddy Yankee
3,Cuéntale,Ivy Queen,"12 ContributorsDale Don Dale Lyrics[Letra de ""...",Dale Don Dale,Don Omar,Dale Don Dale by Don Omar
4,Cuéntale,Ivy Queen,"8 ContributorsSexy Robótica Lyrics[Letra de ""S...",Sexy Robotica,Don Omar,Sexy Robotica by Don Omar


In [4]:
lyrics, song_title = similar_song_lyrics["lyrics"].tolist(), similar_song_lyrics["similar_song"].tolist()
processed_lyrics = []
for lyric in lyrics:
    processed_lyrics.append(nnm.preprocess_sentence(lyric))

In [5]:
#Create word embeddings 
embeddings = embedder(processed_lyrics, batch_size=NUM_SEQUENCES_PER_BATCH)
print(embeddings.shape)


torch.Size([7047, 384])


In [6]:
label_encoder = LabelEncoder()

# Fit and transform the labels to integers
y_encoded = label_encoder.fit_transform(song_title) # Label Encoder does transform values into unique

# Convert to a PyTorch tensor
y_tensor = torch.tensor(y_encoded, dtype=torch.int64).to(device)
print(y_tensor[:5])

tensor([ 734,  487,  863,  655, 2482], device='cuda:0')


In [7]:
model, test_dataloader = nnm.full_pipeline(x=embeddings, y=y_tensor, vocab_size = len(set(y_encoded)), batch_size=NUM_SEQUENCES_PER_BATCH, embedding_size=embeddings.shape[1], hidden_units=[1024, 128], epochs=500, device=device, early_stop_threshold=1e-3)
print(embeddings.shape)  # Should be [num_samples, embedding_dim]

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0, Loss: 8.1028

Epoch: 1, Loss: 7.9375

Epoch: 2, Loss: 7.5144

Epoch: 3, Loss: 7.2940

Epoch: 4, Loss: 7.1606

Epoch: 5, Loss: 7.0159

Epoch: 6, Loss: 6.9943

Epoch: 7, Loss: 6.9276

Epoch: 8, Loss: 6.8484

Epoch: 9, Loss: 6.6928

Epoch: 10, Loss: 6.7115

Epoch: 11, Loss: 6.5226

Epoch: 12, Loss: 6.4915

Epoch: 13, Loss: 6.3719

Epoch: 14, Loss: 6.2226

Epoch: 15, Loss: 6.0835

Epoch: 16, Loss: 5.7843

Epoch: 17, Loss: 5.5242

Epoch: 18, Loss: 5.4079

Epoch: 19, Loss: 4.9902

Epoch: 20, Loss: 4.8106

Epoch: 21, Loss: 4.7285

Epoch: 22, Loss: 4.4814

Epoch: 23, Loss: 4.1925

Epoch: 24, Loss: 4.0046

Epoch: 25, Loss: 3.7542

Epoch: 26, Loss: 3.3250

Epoch: 27, Loss: 3.3113

Epoch: 28, Loss: 2.9351

Epoch: 29, Loss: 2.7957

Epoch: 30, Loss: 2.5110

Epoch: 31, Loss: 2.2890

Epoch: 32, Loss: 2.1544

Epoch: 33, Loss: 2.1637

Epoch: 34, Loss: 1.9587

Epoch: 35, Loss: 1.5996

Epoch: 36, Loss: 1.4843

Epoch: 37, Loss: 1.4224

Epoch: 38, Loss: 1.4019

Epoch: 39, Loss: 1.2328

Epoch: 40,

In [8]:
# Evaluate the model on the test set
accuracy = nnm.evaluate_model(model, test_dataloader)
# Save the trained model
torch.save(model.state_dict(), "model/trained_ffnn_model.pth")
print("Model saved as 'trained_ffnn_model.pth'")
# Load the model for future use
print(accuracy)

Accuracy on test set: 67.09% (946/1410)
Model saved as 'trained_ffnn_model.pth'
0.6709219858156028


In [ ]:
import lyricsgenius
song_title = "Yonaguni"
artist_name = "Bad Bunny"

genius = lyricsgenius.Genius("Z_wiD32yFUiTd_bFET7Xo4UkbdCvZZJm7ViomZFISpf4wpr_4nvblXiuMaemGED8")
queried_song = genius.search_song(song_title, artist_name)

if queried_song:
    lyrics = queried_song.lyrics
    processed_lyrics = nnm.preprocess_sentence(lyric)
    encoded_lyrics = embedder(processed_lyrics, batch_size=NUM_SEQUENCES_PER_BATCH)
    encoded_lyrics.unsqueeze_(0) # Add a batch dimension

    model.eval()

    with torch.no_grad(): # Speeds up inference and reduces memory usage by not having to calcualte gradients
        logits = model(encoded_lyrics) # Forward pass on the model
        probability = nn.functional.softmax(logits, dim=1) # Normalize z scores to probability
        predicted_idx = torch.multinomial(probability, num_samples=10).cpu()[0].squeeze(0) # Get the top 10 predictions
        predicted_songs = label_encoder.inverse_transform(predicted_idx)
        print(probability.cpu()[0]dd[predicted_idx])
        print(predicted_songs)

else:
    print(f"Song '{song_title}' by '{artist_name}' not found.")


Searching for "Yonaguni" by Bad Bunny...
Done.
tensor([9.9971e-01, 9.2111e-05, 1.2355e-05, 4.3078e-05, 1.3216e-05, 2.6250e-05,
        1.2035e-05, 3.5109e-05, 1.3134e-05, 1.4382e-05])
['Poker Face by Lady Gaga' 'Casual by Chappell Roan'
 'Californication by Red Hot Chili Peppers' 'Suga Suga by Baby Bash'
 'Toxic by Britney Spears' 'Sexy Love by Ne-Yo' 'Adorn by Miguel'
 'Nasty by Tinashe' 'Trust! by Rebecca Black' 'Need To Know by Doja Cat']
